In [2]:
from dotenv import load_dotenv

from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

import os
import asyncio


load_dotenv(override=True)
openai_api_key = os.environ["OPENAI_API_KEY"]
twilio_api_sid = os.environ["TWILIO_API_SID"]
twilio_api_secret = os.environ["TWILIO_API_SECRET"]
#twilio_phone_number = os.environ["TWILIO_PHONE_NUMBER"]

In [34]:
agent = Agent(name="thang_he", instructions="you are a rude joke teller", model="gpt-4o-mini")
agent

Agent(name='thang_he', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='you are a rude joke teller', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [ ]:
## Test agent

In [19]:
result = await Runner.run(agent, "tell a joke about schoolbus driver")
print (result.final_output)

## Simple multi agent orchestration

In [37]:
instruction1 = "you are a sales agent working for ComplAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
power y AI. You write professional, serios, cold email"

instruction2 = "you are a humorous, engaging sales agent working for ComlAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
You write witty, engaging cold emails that are likely to get a response"

instruction3 = "you are a busy sales agent working for ComlAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
You write concise, to the point cold email."

In [38]:
sales_agent1 = Agent(name="Professional Sales Agent", instructions=instruction1, model="gpt-4o-mini")
sales_agent2 = Agent(name="Professional Sales Agent", instructions=instruction2, model="gpt-4o-mini")
sales_agent3 = Agent(name="Professional Sales Agent", instructions=instruction3, model="gpt-4o-mini")

# Write email in streaming mode (get returned token by token)

In [40]:
result = Runner.run_streamed(sales_agent1, input="write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data,  ResponseTextDeltaEvent):
        print (event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well.

At ComplAI, we understand that navigating the complexities of SOC 2 compliance can be a daunting task for many organizations. That’s why we’ve developed a comprehensive SaaS solution designed to streamline the compliance process and prepare your team for audit success.

Our platform leverages advanced AI capabilities to automate documentation, streamline workflows, and provide real-time insights, ensuring that your organization remains compliant without the overwhelming burden of manual processes. 

Key benefits of our solution include:

- **Automated Compliance Tracking**: Stay on top of your compliance status with minimal effort.
- **Efficient Preparation for Audits**: Seamless integration of audit-ready documentation and reporting.
- **Real-Time Insights**: Get immediate access to compliance metrics, allowing for proactive adjustments.

I would love to schedule a brie

# Use tracem

In [ ]:
message = "write a cold sales email"
with trace("Parallel cold emaiols"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),  
        Runner.run(sales_agent3, message),  
    
    )
output = [result.final_output for result in results]


In [ ]:
output

["Subject: Streamline Your SOC 2 Compliance with ComplAI\n\nDear [Recipient's Name],\n\nI hope this message finds you well. My name is [Your Name], and I am a representative of ComplAI, a leading SaaS provider specializing in automating SOC 2 compliance processes.\n\nAchieving and maintaining SOC 2 compliance can be daunting and time-consuming. At ComplAI, we understand the challenges organizations face, and we've developed a solution powered by AI to simplify this journey. Our platform not only streamlines the compliance process but also enhances your preparedness for audits, ultimately saving you time and resources.\n\nKey benefits of our solution include:\n- Automated documentation and evidence gathering\n- Real-time compliance monitoring\n- Detailed reporting to support audit readiness\n- Tailored compliance frameworks specific to your industry\n\nI would love the opportunity to discuss how ComplAI can help your organization achieve seamless compliance and mitigate risk. Are you av

#  use email picker to select email

In [ ]:
with trace("Parralel Generate - Select emails"):
    message = "write a cold sales email"
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),  
        Runner.run(sales_agent3, message),  
    
    )
    output = [result.final_output for result in results]

    input_emails = "\n".join(output)
    instruction_picker = "You pick the best cold sales email from the given options," \
    "Imagine you are a customer and pick the one you are likely respond to." \
    "Do not give explanation; reply with the index of the selected email only, index start at 1"
    sales_picker = Agent(name="Email picker", instructions=instruction_picker, model="gpt-4o-mini")
    best = await Runner.run(sales_picker, input_emails)

    print (f"Best email is #{best.final_output}")
    print ("Best email content", "\n", output[int(best.final_output)])

Best email is #2


Best email content 
 Subject: Simplify Your SOC 2 Compliance Process

Hi [First Name],

I hope this message finds you well! I’m reaching out to introduce ComlAI, a SaaS solution designed to streamline your SOC 2 compliance journey.

Our platform helps businesses like yours automate documentation, manage audits, and enhance security measures—all while saving you time and reducing stress.

Would you be open to a quick call next week to discuss how we can support your compliance efforts?

Best regards,  
[Your Name]  
[Your Position]  
ComlAI  
[Your Phone Number]  
[Your LinkedIn Profile]  


In [75]:
tool_sale = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool_sale

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x718dcd2085e0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)